* Show picture of Manhattan distance, like walking a city block
* Show picture of Euclidean distance, talking the "direct path"
* Explain formula for Minkowski distance and break it down
* Show how this works for two points in 2D:
    * For Manhattan distance, we're travelling along the difference on the x-axis and then the distance on the y-axis
    * For Euclidean distance, imagine we treat those two x- and y-differences as sides of a triangle, then calculate the hypotenuse.

While, again, we can only really comfortably visualise this in 2- and 3D, the logic extends to n-dimensions as with vector spaces.

We now know what we need to calculate the Minowski difference, so let's create a simple implementation using loops.

Calculation of Minkowski distance:

Explain what this is in 2-d space:
* Manhattan
* Euclidean

Then show calculation of this using loops

Then work out which linear algebra functions you use to create the optimisation and explain them
Then show LA optimisation + timing for various n-dimensions and datasets
Split into 2 different functions and get rid of sq forloop for Manhattan and abs forloop for Euclidean
Comparing timing to sklearn implementation

# Just enough linear algebra to optimise the Minkowski distance

In the last blog post, we discussed how to calculate the Manhattan and Euclidean distances from first principles. However, in that post, we did a very manual implementation for a single pair of vectors, which would not generalise well to more than one pair and would become cumbersome for more than a few dimensions. As such, in this blog post we will discuss how to implement the Minkowski distance calculation for vectors of any number of dimensions and for any numbers of pairs. As doing pairwise calculations can become very computationally expensive, we will also discuss how to optimise our distance calculations using some further knowledge from linear algebra.

## Our inital attempt
Let's start by revisiting our manual implementation of the Euclidean distance:

In [1]:
import numpy as np

X = [5, 7, 3, 9]
Y = [1, 6, 2, 4]
i = len(X)
p = 2

diff_1 = np.abs(X[0] - Y[0]) ** p
diff_2 = np.abs(X[1] - Y[1]) ** p
diff_3 = np.abs(X[2] - Y[2]) ** p
diff_4 = np.abs(X[3] - Y[3]) ** p

euclidean_distance_sq = diff_1 + diff_2 + diff_3 + diff_4
euclidean_distance = euclidean_distance_sq ** (1 / p)
euclidean_distance

6.557438524302

Right away, we can see one possibility for generalising our code over as many dimensions as we like: we can run all of our absolute difference calculations in a loop. Let's see how this looks:

In [2]:
diffs = []
for element in np.arange(0, i):
    diffs.append(np.abs(X[element] - Y[element]) ** p)

We now have a list containing the distance metrics, which we can then add.

In [3]:
euclidean_distance_sq = sum(diffs)

Let's bundle this up into our first Minkowski distance function.

In [4]:
def calculate_minkowski_distance(X, Y, p):
    """Calculates the Minkowski distance between two vectors, X and Y. When p = 1, calculates the Manhattan distance, when p = 2, calculates the Euclidean distance."""
    i = len(X)
    diffs = []
    for element in np.arange(0, i):
        diffs.append(np.abs(X[element] - Y[element]) ** p)
    euclidean_distance_sq = sum(diffs)
    return euclidean_distance_sq ** (1 / p)

If we apply it to our two test vectors, we can see we get the exact same answer:

In [5]:
calculate_minkowski_distance(X, Y, 2)

6.557438524302

We now need a way of applying our newly minted Minkowski distance function to multiple pairs at the same time. What we need to do is compare every vector all other vectors pairwise, therefore the simplest way of doing this is within a nested loop. We'll create an additional vector, $Z$ to compare to $X$ and $Y$. We then combine this into a list of lists along with $X$ and $Y$.

In [6]:
Z = [8, 8, 3, 1]
vectors = [X, Y, Z]

What we'll now do is loop over each vector in this list, comparing $X$ with itself, $X$ with $Y$, etc., using two nested forloops.

In [7]:
distances = []
for a in vectors:
    tmp_distances = []
    for b in vectors:
        tmp_distances.append(calculate_minkowski_distance(a, b, 2))
    distances.append(tmp_distances)

Finally, we can output the pairwise distance matrix to a Pandas DataFrame.

In [8]:
import pandas as pd
pd.DataFrame(distances)

,0,1,2
0,0.000000,6.557439,8.602325
1,6.557439,0.000000,7.937254
2,8.602325,7.937254,0.000000


We can see that everything makes sense: vectors compared with themselves have a distance of 0, and we can see the known distance between $X$ and $Y$ is still returned as 6.56. Let's now package this up into a function and see how it performs on when calculating pairwise differences on our beans dataset.

In [9]:
def apply_minkowski_distance(vectors: list, p: int) -> pd.DataFrame:
    distances = []
    for a in vectors:
        tmp_distances = []
        for b in vectors:
            tmp_distances.append(calculate_minkowski_distance(a, b, p))
        distances.append(tmp_distances)
    return pd.DataFrame(distances)

As a refresher for those of you who may not have read the last blog post, the [dry bean dataset](https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset#) describes the characteristics of 13,611 images of dried beans across 7 different types. There are 16 features describing the beans, such as their area, aspect ratio and roundness. We'll prepare three samples for testing using the Euclidean distance:
* A sample of 100 observations using only three of the features (`MajorAxisLength`, `MinorAxisLength` and `roundness`);
* All observations using only the above three features; and
* The entire dataset with all features.

As our function above takes in a list of lists containing each of the vectors, we convert our Pandas DataFrames to this format by first extracting the values of each row using `values` and then converting this to a list using `tolist()`.

In [10]:
beans = pd.read_excel("data/Dry_Bean_Dataset.xlsx")

list_sample_1 = beans[["MajorAxisLength", "MinorAxisLength", "roundness"]][:100].values.tolist()
list_sample_2 = beans[["MajorAxisLength", "MinorAxisLength", "roundness"]].values.tolist()
list_sample_3 = beans.drop(columns = ["Class"]).values.tolist()

We then run the pairwise Euclidean distance calculation over each of our samples.

In [25]:
%time list_min_1 = apply_minkowski_distance(list_sample_1, 2)

CPU times: user 84.5 ms, sys: 34.2 ms, total: 119 ms
Wall time: 89.4 ms


Our small sample with three features finishes relatively quickly.

In [12]:
%time list_min_2 = apply_minkowski_distance(list_sample_2, 2)

CPU times: user 16min 47s, sys: 18.2 s, total: 17min 5s
Wall time: 16min 56s


However, when we move up to 100 times that many observations, our operation takes more than 10,000 times as long to complete, clocking in at an eyewatering 17 minutes. It's easy to see why when we look at the code: in our nested forloop we're multiplying every single element by every other element (including itself), meaning that the number of operations we need to complete is the square of the number of vectors. You can confirm this by seeing that our example vector containing $X$, $Y$ and $Z$: we had three vectors, but ended up with 9 calculations in our distance matrix.

In [30]:
%time list_min_3 = apply_minkowski_distance(list_sample_3, 2)

CPU times: user 1h 5min 48s, sys: 2min 15s, total: 1h 8min 4s
Wall time: 1h 6min 37s


We can see that our processing time has just increased approximately fourfold. This makes sense given that we're looping over each element in the vector to get the difference scores needed to calculate our Euclidean distance. As we've jumped from 3 to 16 features, we'd expect a corresponding increase in the processing time for each distance calculation. This calculation of the difference score element-by-element is the first inefficiency we'll tackle. To understand how to do so, let's have a discussion about some of the arithmetic we can do with vectors.

## Vector arithmetic

Just like with numbers, we can do some simple arithmetic operations with vectors. The first of these is that we can take two vectors with the same number of elements and subtract them. This results in the corresponding elements being lined up and the second subtracted from the first. Let's see an example.

In [11]:
Xm = np.array([5, 7, 3, 9])
Ym = np.array([1, 6, 2, 4])

Ym - Xm

array([-4, -1, -1, -5])

We can see that by subtracting $Xm$ from $Ym$, we ended up with pairwise subtraction of each element of each, with $Ym_1 - Xm_1 = 1 - 5 = -4$, $Ym_2 - Xm_2 = 6 - 7 = -1$, and so on. You can probably see already that if we can replace the forloop to calculate the difference elementwise in our Minkowski function, we could save quite a lot of processing time.

In order to fully replace this functionality, we need to know about two other things we can do with vectors. Using `numpy`, we can apply an [absolute difference function](https://numpy.org/doc/stable/reference/generated/numpy.absolute.html) to each element in the vector. In addition, we can raise each element in a vector to a power (also known as the Hadamard power) using `numpy`'s `power` method. Let's see these both in action.

In [12]:
np.abs(Ym - Xm)

array([4, 1, 1, 5])

You can see it has worked, with all values now converted to positive. Let's now apply the `power` function:

In [13]:
np.power(np.abs(Ym - Xm), 2)

array([16,  1,  1, 25])

We've now replicated all of the functionality we were including in the loop in our Minkowski function. Let's update it and see whether our performance has improved.

In [14]:
def calculate_minkowski_distance(X, Y, p):
    """Calculates the Minkowski distance between two vectors, X and Y. When p = 1, calculates the Manhattan distance, when p = 2, calculates the Euclidean distance."""
    diffs = np.power(np.abs(X - Y), p)
    euclidean_distance_sq = sum(diffs)
    return euclidean_distance_sq ** (1 / p)

As a quick sanity check, let's confirm that we get the same answer as with our previous version of the function.

In [15]:
calculate_minkowski_distance(Xm, Ym, 2)

6.557438524302

Having confirmed this, let's test it out with the same samples as previously. We now need to convert these into numpy arrays to be able to exploit the `numpy` vector functions we're using.

In [16]:
array_sample_1 = beans[["MajorAxisLength", "MinorAxisLength", "roundness"]][:100].to_numpy()
array_sample_2 = beans[["MajorAxisLength", "MinorAxisLength", "roundness"]].to_numpy()
array_sample_3 = beans.drop(columns = ["Class"]).to_numpy()

Let's now test out how it performs compared to our last implementation.

In [17]:
%time list_min_1 = apply_minkowski_distance(array_sample_1, 2)

CPU times: user 52.5 ms, sys: 2.74 ms, total: 55.3 ms
Wall time: 53.6 ms


In [18]:
%time list_min_2 = apply_minkowski_distance(array_sample_2, 2)

CPU times: user 13min 1s, sys: 6.9 s, total: 13min 8s
Wall time: 13min 10s


In [19]:
%time list_min_3 = apply_minkowski_distance(array_sample_3, 2)

CPU times: user 17min 16s, sys: 7.35 s, total: 17min 23s
Wall time: 17min 26s


We can see that we're getting significantly faster times, especially

In [22]:
vectors = np.array([
    np.array([5, 7, 3, 9]),
    np.array([1, 6, 2, 4]),
    np.array([8, 8, 3, 1])
])

diffs = vectors[:, None, :] - vectors[None, :, :]
np.abs(diffs)**2

array([[[ 0,  0,  0,  0],
        [16,  1,  1, 25],
        [ 9,  1,  0, 64]],

       [[16,  1,  1, 25],
        [ 0,  0,  0,  0],
        [49,  4,  1,  9]],

       [[ 9,  1,  0, 64],
        [49,  4,  1,  9],
        [ 0,  0,  0,  0]]])

 * Time this on beans dataset (different subset: small with 3 features, all with 3 features, all with all features)
    * 100, 500, 1000, 5000 and 10000 observations with all 16 features
    * 1 through to 16 features with 1000 observations
* Implement with vector subtraction but still looping (explain relevant linear algebra)
* Time this
* Implement with broadcasting to get rid of final loop (explain relevant numpy function)
* Time this
* Compare with sklearn



##
* Vector subtraction
    * Broadcasting vector subtraction (to do this efficiently)
* Squaring a vector
*



In [14]:
def calculate_minkowski_distance_vected(array1: np.ndarray,
                                        array2: np.ndarray,
                                        p: int
                                        ) -> np.ndarray:
    """
    Generalised formula for calculating both Manhattan and Euclidean distances. Calculates pairwise distances between every point in two n-dimensional arrays.
    array1: first set of points;
    array2: second set of points;
    p: power parameter which determines the distance metric used, with 1 = Manhattan and 2 = Euclidean.
    """

    diffs = array1[:, None, :] - array2[None, :, :]
    abs_diffs = (np.abs(diffs) ** p)
    return abs_diffs.sum(axis=-1) ** (1 / p)

In [15]:
array_sample_1 = beans[["MajorAxisLength", "MinorAxisLength", "roundness"]][:100].to_numpy()
array_sample_2 = beans[["MajorAxisLength", "MinorAxisLength", "roundness"]].to_numpy()
array_sample_3 = beans.drop(columns = ["Class"]).to_numpy()

In [16]:
%time array_euclidean_1 = calculate_minkowski_distance_vected(array_sample_1, array_sample_1, 2)

CPU times: user 786 µs, sys: 838 µs, total: 1.62 ms
Wall time: 915 µs


In [17]:
%time array_euclidean_2 = calculate_minkowski_distance_vected(array_sample_2, array_sample_2, 2)

CPU times: user 3.18 s, sys: 2.19 s, total: 5.37 s
Wall time: 5.38 s


In [18]:
%time array_euclidean_3 = calculate_minkowski_distance_vected(array_sample_3, array_sample_3, 2)

CPU times: user 31.7 s, sys: 2min 10s, total: 2min 42s
Wall time: 5min 16s
